<a href="https://colab.research.google.com/github/mustafaakben/QualtricsGeoLocation/blob/main/src/GeoLocationFull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Upload Qualtrics csv data and you will get the results {display-mode: "form"}
# This code will be hidden when the notebook is loaded.

import numpy as np
import time
import pandas as pd
import os
import requests

def get_ip_location(ip_address, retries=10, backoff_factor=2):
    for i in range(retries):
        try:
            response = requests.get(f"http://ip-api.com/json/{ip_address}")

            if response.status_code == 200:  # Success
                js = response.json()
                if js['status'] == 'fail':
                    print(f"Unable to get location for IP Address: {ip_address}")
                else:
                    print(f"IP Address: {ip_address}")
                    print(f"Country: {js['country']}")
                    print(f"Region: {js['regionName']}")
                    print(f"City: {js['city']}")
                    print(f"Zip: {js['zip']}")
                    print(f"Latitude: {js['lat']}")
                    print(f"Longitude: {js['lon']}")
                    print(f"ISP: {js['isp']}")
                    print(f"Organisation: {js['org']}")
                    print(f"AS: {js['as']}")
                    return js
                break
            elif response.status_code == 429:  # Too Many Requests
                print(f"Rate limit exceeded. Retrying in {backoff_factor}s...")
                time.sleep(backoff_factor)  # Sleep for backoff_factor seconds
                backoff_factor *= 2  # Exponential backoff
            else:
                print(f"Unexpected status code {response.status_code}. Retrying in {backoff_factor}s...")
                time.sleep(backoff_factor)  # Sleep for backoff_factor seconds
                backoff_factor *= 2  # Exponential backoff
        except Exception as e:
            print(f"An error occurred: {str(e)}. Retrying in {backoff_factor}s...")
            time.sleep(backoff_factor)  # Sleep for backoff_factor seconds
            backoff_factor *= 2  # Exponential backoff


from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


## Upload Data
files_names = os.listdir()
csvfiles = [files for files in files_names if files.endswith(".csv")]
file_path = csvfiles[0]
data = pd.read_csv(file_path)

# Check whether the first and second rows are string or numeric
IPs = data['IPAddress']
drop_rows = IPs.isin(["IP Address","""{"ImportId":"ipAddress"}"""])
data = data[~drop_rows]
data.reset_index(inplace=True, drop=True)
IPs = data['IPAddress']
GeoLocations_reported = data[['LocationLatitude','LocationLongitude']]


results = []
for ip in IPs:
    # time.sleep(.8)
    results.append(get_ip_location(ip))
    print("*"*10)

geoLocation_valid = []
for res in results:
    if res is not None:
        geoLocation_valid.append((res['lat'], res['lon']))
    else:
        geoLocation_valid.append((0,0))

geo_location_information = []
geoLocation_valid = []
for res in results:
    if res is not None:
        geoLocation_valid.append((res['lat'], res['lon']))
        geo_location_information.append((res['country'],res['city']))
    else:
        geoLocation_valid.append((0,0))
        geo_location_information.append((np.nan,np.nan))

GeoLocations_validated = pd.DataFrame(geoLocation_valid, columns=['LocationLatitude', 'LocationLongitude'])
GeoLocation_information = pd.DataFrame(geo_location_information,columns= ["Country","City"])
GeoLocations_validated = pd.DataFrame(geoLocation_valid, columns=['LocationLatitude', 'LocationLongitude'])

lat_val = GeoLocations_validated['LocationLatitude'].values
lon_val = GeoLocations_validated['LocationLongitude'].values

lat_rep = GeoLocations_reported['LocationLatitude'].values.astype(float)
lon_rep = GeoLocations_reported['LocationLongitude'].values.astype(float)

GeoLocations_validated = pd.DataFrame(geoLocation_valid, columns=['LocationLatitude', 'LocationLongitude'])
lat_val = GeoLocations_validated['LocationLatitude'].values
lon_val = GeoLocations_validated['LocationLongitude'].values
lat_rep = GeoLocations_reported['LocationLatitude'].values.astype(float)
lon_rep = GeoLocations_reported['LocationLongitude'].values.astype(float)
latitude_res = (lat_val == lat_rep)
longitude_res = (lon_val == lon_rep)


geo_location_combined = pd.DataFrame((lat_val, lat_rep, latitude_res, lon_val, lon_rep, longitude_res)).T
geo_location_combined.columns = ["Latitude_Validated", "Latitude_Reported", "Latitude_Match", "Longitude_Validated", "Longitude_Reported", "Longitude_Match"]
geo_location_combined['absDeviation'] = abs(geo_location_combined['Latitude_Validated'] - geo_location_combined['Latitude_Reported']) + abs(geo_location_combined['Longitude_Validated'] - geo_location_combined['Longitude_Reported'])
# Column order absDeviation first, and then the rest

geo_location_combined = geo_location_combined[["absDeviation", "Latitude_Validated", "Latitude_Reported", "Latitude_Match", "Longitude_Validated", "Longitude_Reported", "Longitude_Match"]]
# Concatenate the original data with the validated data
data = pd.concat([geo_location_combined,GeoLocation_information,data], axis=1)
data.sort_values(by=['absDeviation'], ascending=False, inplace=True)
data.to_csv("data_validated.csv", index=False)
from google.colab import files
files.download('/content/data_validated.csv')



Saving Absurdity+EMPLOYEE+REGISTRATION_October+5,+2023_17.18.csv to Absurdity+EMPLOYEE+REGISTRATION_October+5,+2023_17.18.csv
User uploaded file "Absurdity+EMPLOYEE+REGISTRATION_October+5,+2023_17.18.csv" with length 67355 bytes
Unable to get location for IP Address: nan
**********
IP Address: 184.89.166.246
Country: United States
Region: Florida
City: Altamonte Springs
Zip: 32714
Latitude: 28.6623
Longitude: -81.4136
ISP: Charter Communications
Organisation: Spectrum
AS: AS33363 Charter Communications, Inc
**********
IP Address: 203.164.251.187
Country: Australia
Region: New South Wales
City: Sydney
Zip: 2000
Latitude: -33.8715
Longitude: 151.2006
ISP: OPTUSINTERNET
Organisation: 
AS: AS4804 Microplex PTY LTD
**********
IP Address: 203.164.251.187
Country: Australia
Region: New South Wales
City: Sydney
Zip: 2000
Latitude: -33.8715
Longitude: 151.2006
ISP: OPTUSINTERNET
Organisation: 
AS: AS4804 Microplex PTY LTD
**********
IP Address: 137.215.239.236
Country: South Africa
Region: Gaut

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>